In [1]:
import random
import json
import numpy as np
import pickle

import nltk
from nltk.stem import WordNetLemmatizer

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation,Dropout
from tensorflow.keras.optimizers import SGD
from tensorflow.python.keras.backend import dropout

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [3]:
lemmatizer = WordNetLemmatizer()

In [12]:
intents = json.loads(open('intents.json').read())

In [13]:
intents

{'intents': [{'tag': 'greetings',
   'patterns': ['hello',
    'hi',
    'hey',
    'good day',
    'greetings',
    "what's up?",
    'how is it going?'],
   'responses': ['Hello!', 'Hey!', 'What can I do for you?']},
  {'tag': 'bye',
   'patterns': ['bye bye', 'goodbye', 'see you later', 'see ya'],
   'responses': ['Goodbye!', 'Come back soon!', 'See ya! Will miss you']},
  {'tag': 'happy',
   'patterns': ['good day', 'awesome', 'excellent', 'amazing', 'nice'],
   'responses': ['wow thats great!', 'I am happy for you', 'Nicee']},
  {'tag': 'sad',
   'patterns': ['not good', 'okish', 'bad', 'worst', 'frown'],
   'responses': ['well...cheer up! Just as master Oogway says, "There are no accidents" ']}]}

In [14]:
words = []
classes = []
documents = []
ignore_letters =['?','!',".",","]

for intent in intents['intents']:
    for pattern in intent['patterns']:
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)
        documents.append((word_list,intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [15]:
words

['hello',
 'hi',
 'hey',
 'good',
 'day',
 'greetings',
 'what',
 "'s",
 'up',
 '?',
 'how',
 'is',
 'it',
 'going',
 '?',
 'bye',
 'bye',
 'goodbye',
 'see',
 'you',
 'later',
 'see',
 'ya',
 'good',
 'day',
 'awesome',
 'excellent',
 'amazing',
 'nice',
 'not',
 'good',
 'okish',
 'bad',
 'worst',
 'frown']

In [16]:
classes

['greetings', 'bye', 'happy', 'sad']

In [17]:
documents

[(['hello'], 'greetings'),
 (['hi'], 'greetings'),
 (['hey'], 'greetings'),
 (['good', 'day'], 'greetings'),
 (['greetings'], 'greetings'),
 (['what', "'s", 'up', '?'], 'greetings'),
 (['how', 'is', 'it', 'going', '?'], 'greetings'),
 (['bye', 'bye'], 'bye'),
 (['goodbye'], 'bye'),
 (['see', 'you', 'later'], 'bye'),
 (['see', 'ya'], 'bye'),
 (['good', 'day'], 'happy'),
 (['awesome'], 'happy'),
 (['excellent'], 'happy'),
 (['amazing'], 'happy'),
 (['nice'], 'happy'),
 (['not', 'good'], 'sad'),
 (['okish'], 'sad'),
 (['bad'], 'sad'),
 (['worst'], 'sad'),
 (['frown'], 'sad')]

In [18]:
words = [lemmatizer.lemmatize(word) for word in words if word not in ignore_letters]
words = sorted(set(words))

In [19]:
words

["'s",
 'amazing',
 'awesome',
 'bad',
 'bye',
 'day',
 'excellent',
 'frown',
 'going',
 'good',
 'goodbye',
 'greeting',
 'hello',
 'hey',
 'hi',
 'how',
 'is',
 'it',
 'later',
 'nice',
 'not',
 'okish',
 'see',
 'up',
 'what',
 'worst',
 'ya',
 'you']

In [22]:
classes = sorted(set(classes))
pickle.dump(words, open('words.pkl','wb'))
pickle.dump(classes , open('classes.pkl','wb'))

In [21]:
classes

['bye', 'greetings', 'happy', 'sad']

In [23]:
training = []
output_empty = [0] * len(classes)

In [30]:
len(output_empty)

4

In [31]:
for document in documents:
    bag = []
    word_patterns = document[0]
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)
        
    output_row = list(output_empty)
    output_row[classes.index(document[1])] = 1
    training.append([bag, output_row])

In [33]:
training[1]

[[0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [0, 1, 0, 0]]